The goal of this CNN is to classify wind faults into four categories: 
1. 3 Phase to Ground
2. 2 Phase line to line
3. 2 phase to earth
4. 1 pole line to ground. 



In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

Preprocessing + Rolling Windows

In [3]:
WINDOW_SIZE = 100  # 1 second window (adjust if needed)
LABEL_MAPPING = {
    "3P.GND": 0,  # 3-phase to ground
    "2P.LL": 1,   # 2-phase line to line
    "2P.EARTH": 2, # 2-phase to earth
    "1P.GND": 3    # 1-pole line to ground
}
class WindFaultDataset(Dataset):
    """ PyTorch Dataset for Wind Fault Classification """
    def __init__(self, file_paths):
        self.data = []
        self.labels = []
        self.scaler = MinMaxScaler()

        for file_path in file_paths:
            features, label = self.process_csv(file_path)
            windows, labels = self.create_rolling_windows(features, label, WINDOW_SIZE)
            self.data.append(windows)
            self.labels.append(labels)

        self.data = np.vstack(self.data)  # Stack all windows
        self.labels = np.hstack(self.labels)

    def process_csv(self, file_path):
        """ Load and preprocess a CSV file. """
        df = pd.read_csv(file_path)  # Skip headers
        df.columns = pd.read_csv(file_path, nrows=1).iloc[0]  # Assign proper column names
        df = df.apply(pd.to_numeric, errors='coerce')  # Convert to numeric
        
        # Normalize features
        feature_data = self.scaler.fit_transform(df.iloc[:, 1:])  # Exclude time column

        # Extract label from filename
        fault_type = os.path.basename(file_path).split("-")[0]
        label = LABEL_MAPPING.get(fault_type, -1)

        return feature_data, label

    def create_rolling_windows(self, data, label, window_size):
        """ Generate overlapping rolling windows of data. """
        windows = []
        labels = []
        for i in range(len(data) - window_size + 1):  # Overlapping windows
            windows.append(data[i:i + window_size])
            labels.append(label)
        return np.array(windows), np.array(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

In [4]:
file_paths = ["Wind_Faults\\1P-60-0-0.csv", "Wind_Faults\\2P-40-20-0.csv"]  # Replace with actual file paths
dataset = WindFaultDataset(file_paths)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Print dataset shape
sample_data, sample_label = dataset[0]
print("Sample data shape:", sample_data.shape)  # (WINDOW_SIZE, num_features)
print("Sample label:", sample_label)

Sample data shape: torch.Size([100, 18])
Sample label: tensor(-1)


Friday: Add in the other two components from ChatGPT